In [1]:
import os
import numpy as np

import tensorflow as tf
from tensorflow.python.tools import strip_unused_lib
from tensorflow.python.framework import dtypes
from tensorflow.python.platform import gfile

import tfcoreml
import coremltools
from coremltools.models import datatypes
from coremltools.models import neural_network
from coremltools.models.pipeline import *
from coremltools.proto import FeatureTypes_pb2


print("tf version", tf.__version__)
print("coremltools version", coremltools.__version__)

W0910 12:10:20.393846 4512996800 __init__.py:74] TensorFlow version 1.15.0 detected. Last version known to be fully compatible is 1.14.0 .
W0910 12:10:20.466037 4512996800 __init__.py:128] Keras version 2.3.1 detected. Last version known to be fully compatible of Keras is 2.2.4 .


tf version 1.15.0
coremltools version 3.4


In [2]:
object_class = 153

In [25]:
def load_saved_model(path):
    """Loads a saved model into a graph."""
    print("Loading saved_model.pb from '%s'" % path)
    the_graph = tf.Graph()
    with tf.Session(graph=the_graph) as sess:
        tf.saved_model.loader.load(sess, [tf.saved_model.tag_constants.SERVING], path)
    return the_graph

# the_graph = load_saved_model("/Users/kehwaweng/Documents/ObjectDetection/create_custom_dataset/frozen_model_r1_13/saved_model/")
# the_graph = load_saved_model("/Users/kehwaweng/Documents/ObjectDetection/create_custom_dataset/frozen_model/saved_model/")
# the_graph = load_saved_model("/Volumes/IPEVO_X0244/object_detection_dataset/frozen_model_934/saved_model/")
# the_graph = load_saved_model("/Volumes/IPEVO_X0244/object_detection_dataset/frozen_model_v19_d0902/saved_model/")
# the_graph = load_saved_model("/Volumes/IPEVO_X0244/object_detection_dataset/frozen_model_v19_train_to_convert/saved_model/")
the_graph = load_saved_model("/Volumes/IPEVO_X0244/object_detection_dataset/frozen_model_1000000/saved_model/")


Loading saved_model.pb from '/Volumes/IPEVO_X0244/object_detection_dataset/frozen_model_1000000/saved_model/'


In [26]:
def optimize_graph(graph):
    """Strips unused subgraphs and save it as another frozen TF model."""
    frozen_model_file = "frozen_model.pb"

    input_node= "Preprocessor/sub"
    bbox_output_node = "concat"
    class_output_node = "Postprocessor/convert_scores"

    gdef = strip_unused_lib.strip_unused(
            input_graph_def = graph.as_graph_def(),
            input_node_names = [input_node],
            output_node_names = [bbox_output_node, class_output_node],
            placeholder_type_enum = dtypes.float32.as_datatype_enum)

    with gfile.GFile(frozen_model_file, "wb") as f:
        f.write(gdef.SerializeToString())
        
optimize_graph(the_graph)

In [27]:
frozen_model_file = "frozen_model.pb"


input_node = "Preprocessor/sub"
bbox_output_node = "concat"
class_output_node = "Postprocessor/convert_scores"

input_tensor = input_node
bbox_output_tensor = bbox_output_node
class_output_tensor = class_output_node

In [28]:
ssd_model = tfcoreml.convert(tf_model_path = frozen_model_file,
                             minimum_ios_deployment_target = '13',
                             mlmodel_path = "/Volumes/IPEVO_X0244/object_detection_dataset/test.mlmodel",
                             input_name_shape_dict = {input_tensor: [1, 300, 300, 3] },
                             image_input_names = input_tensor,
                             output_feature_names = [bbox_output_tensor, class_output_tensor],                             
                             is_bgr = False,
                             red_bias = -1.0,
                             green_bias = -1.0,
                             blue_bias = -1.0,
                             image_scale = 2. / 255)

0 assert nodes deleted
404 nodes deleted
318 nodes deleted
0 nodes deleted
[Op Fusion] fuse_bias_add() deleted 24 nodes.
4 identity nodes deleted
55 disconnected nodes deleted
[Op Fusion] Fused 456 nodes into BatchNorms.
[SSAConverter] Converting function main ...
[SSAConverter] [1/311] Converting op type: 'Placeholder', name: 'Preprocessor/sub', output_shape: (1, 300, 300, 3).
[SSAConverter] [2/311] Converting op type: 'Const', name: 'FeatureExtractor/MobilenetV2/expanded_conv/depthwise/depthwise_weights/read', output_shape: (3, 3, 32, 1).
[SSAConverter] [3/311] Converting op type: 'Const', name: 'FeatureExtractor/MobilenetV2/expanded_conv_1/depthwise/depthwise_weights/read', output_shape: (3, 3, 96, 1).
[SSAConverter] [4/311] Converting op type: 'Const', name: 'FeatureExtractor/MobilenetV2/expanded_conv_2/depthwise/depthwise_weights/read', output_shape: (3, 3, 144, 1).
[SSAConverter] [5/311] Converting op type: 'Const', name: 'FeatureExtractor/MobilenetV2/expanded_conv_3/depthwise/de

In [29]:
spec = ssd_model.get_spec()

# spec.description.output[0].type.multiArrayType.shape.extend([1,1917,1,4])
# spec.description.output[1].type.multiArrayType.shape.extend([1,1917,199])
# coremltools.utils.save_spec(spec,"/Users/kehwaweng/Documents/ObjectDetection/create_custom_dataset/tmp.mlmodel")


In [30]:

## add squeeze in box predictor
new_layer = spec.neuralNetwork.layers.add()
new_layer.name = "squeeze_concat"
params = new_layer.squeeze
params.axes.append(2)
new_layer.input.append("concat")
new_layer.output.append("squeeze_concat_output")

spec.description.output[0].name = "squeeze_concat_output"

spec.description.output[0].type.multiArrayType.shape.extend([1, 1917, 4])
spec.description.output[0].type.multiArrayType.dataType = FeatureTypes_pb2.ArrayFeatureType.DOUBLE
spec.description.output[1].type.multiArrayType.shape.extend([1, 1917, object_class + 1])
spec.description.output[1].type.multiArrayType.dataType = FeatureTypes_pb2.ArrayFeatureType.DOUBLE


# # And the "boxes" output shape is (4, 1917, 1) so get rid of that last one.
# # del spec.description.output[1].type.multiArrayType.shape[-1]
# spec.description.output[1].type.multiArrayType.shape.append(4)
# spec.description.output[1].type.multiArrayType.shape.append(1917)

# Create a new MLModel from the modified spec and save it.
ssd_model = coremltools.models.MLModel(spec)
ssd_model.save("/Volumes/IPEVO_X0244/object_detection_dataset//test.mlmodel") 

In [31]:

def get_anchors( sess, graph, tensor_name):
    """
    Computes the list of anchor boxes by sending a fake image through the graph.
    Outputs an array of size (4, num_anchors) where each element is an anchor box
    given as [ycenter, xcenter, height, width] in normalized coordinates.
    """
    image_tensor = graph.get_tensor_by_name("image_tensor:0")
    box_corners_tensor = graph.get_tensor_by_name(tensor_name)
    box_corners = sess.run(box_corners_tensor, feed_dict={image_tensor: np.zeros((1, 300, 300, 3))})

    ymin, xmin, ymax, xmax = np.transpose(box_corners)
    width = xmax - xmin
    height = ymax - ymin
    ycenter = ymin + height / 2.
    xcenter = xmin + width / 2.
    return np.stack([ycenter, xcenter, height, width])

anchors_tensor = "Concatenate/concat:0"
with the_graph.as_default():
    with tf.Session(graph = the_graph) as sess:
        anchors = get_anchors( sess, the_graph, anchors_tensor)
        assert(anchors.shape[1] == 1917)

# MLMultiArray inputs of neural networks must have 1 or 3 dimensions. 
# We only have 2, so add an unused dimension of size one at the back.
# input_features = [ ("scores", datatypes.Array(198 + 1, 1917, 1)),
#                    ("boxes", datatypes.Array(4, 1917, 1)) ]


# input_features = [("concat", datatypes.Array(4, 1917,1)),
# input_features = [ ("squeeze_concat_output", datatypes.Array(4,1917,1)),                  
#                   ("Postprocessor/convert_scores", datatypes.Array(199,1917,1))]

input_features = [
    ("squeeze_concat_output", datatypes.Array(1, 1917, 4)),
    ("Postprocessor/convert_scores", datatypes.Array(1, 1917, object_class + 1))
]

# The outputs of the decoder model should match the inputs of the next
# model in the pipeline, NonMaximumSuppression. This expects the number
# of bounding boxes in the first dimension.
output_features = [ ("raw_confidence", datatypes.Array(1917, object_class)),
                    ("raw_coordinates", datatypes.Array(1917, 4)) ]

In [32]:
builder = neural_network.NeuralNetworkBuilder(input_features, output_features)

# (num_classes+1, num_anchors, 1) --> (1, num_anchors, num_classes+1)
# builder.add_permute(name = "permute_scores",
#                     dim = (0, 3, 2, 1),
# #                     input_name = "scores",
#                     input_name = "Postprocessor/convert_scores",
#                     output_name = "permute_scores_output")

##
# confidence
##
# builder.add_permute(name = "permute_scores",
#                     dim = (0, 1, 2, 3),
#                     input_name = "Postprocessor/convert_scores",
#                     output_name = "permute_scores_output")

# Strip off the "unknown" class (at index 0).
builder.add_slice(name = "slice_scores",
                  input_name = "Postprocessor/convert_scores",
#                   input_name = "permute_scores_output",
                  output_name = "raw_confidence",
                  axis = "width",
                  start_index = 1,
                  end_index = object_class + 1)

##
#  bounding box
##
# Grab the y, x coordinates (channels 0-1).


builder.add_permute(name = "permute_boxes",
                    dim = (0, 3, 2, 1),
                    input_name = "squeeze_concat_output",
                    output_name = "permute_boxes_output")
                    
builder.add_slice(name = "slice_yx",
#                   input_name = "boxes",
                  input_name = "permute_boxes_output",
                  output_name = "slice_yx_output",
                  axis = "channel",
                  start_index = 0,
                  end_index = 2)

# boxes_yx / 10
builder.add_elementwise(name = "scale_yx",
                        input_names = "slice_yx_output",
                        output_name = "scale_yx_output",
                        mode = "MULTIPLY",
                        alpha = 0.1)

# Split the anchors into two (2, 1917, 1) arrays.
anchors_yx = np.expand_dims(anchors[:2, :], axis = -1)
anchors_hw = np.expand_dims(anchors[2:, :], axis= -1)

builder.add_load_constant(name = "anchors_yx",
                          output_name = "anchors_yx",
                          constant_value = anchors_yx,
                          shape = [2, 1917, 1])

builder.add_load_constant(name = "anchors_hw",
                          output_name = "anchors_hw",
                          constant_value = anchors_hw,
                          shape = [2, 1917, 1])

# (boxes_yx / 10) * anchors_hw
builder.add_elementwise(name = "yw_times_hw",
                        input_names = ["scale_yx_output", "anchors_hw"],
                        output_name = "yw_times_hw_output",
                        mode = "MULTIPLY")

# (boxes_yx / 10) * anchors_hw + anchors_yx
builder.add_elementwise(name = "decoded_yx",
                        input_names = ["yw_times_hw_output", "anchors_yx"],
                        output_name = "decoded_yx_output",
                        mode = "ADD")

# Grab the height and width (channels 2-3).
builder.add_slice(name = "slice_hw",
#                   input_name = "boxes",
                  input_name = "permute_boxes_output",
                  output_name = "slice_hw_output",
                  axis = "channel",
                  start_index = 2,
                  end_index = 4)

# (boxes_hw / 5)
builder.add_elementwise(name = "scale_hw",
                        input_names = "slice_hw_output",
                        output_name = "scale_hw_output",
                        mode = "MULTIPLY",
                        alpha = 0.2)

# exp(boxes_hw / 5)
builder.add_unary(name = "exp_hw",
                  input_name = "scale_hw_output",
                  output_name = "exp_hw_output",
                  mode = "exp")

# exp(boxes_hw / 5) * anchors_hw
builder.add_elementwise(name = "decoded_hw",
                        input_names = ["exp_hw_output", "anchors_hw"],
                        output_name = "decoded_hw_output",
                        mode = "MULTIPLY")

# The coordinates are now (y, x) and (height, width) but NonMaximumSuppression
# wants them as (x, y, width, height). So create four slices and then concat
# them into the right order.
builder.add_slice(name = "slice_y",
                  input_name = "decoded_yx_output",
                  output_name = "slice_y_output",
                  axis = "channel",
                  start_index = 0,
                  end_index = 1)

builder.add_slice(name = "slice_x",
                  input_name = "decoded_yx_output",
                  output_name = "slice_x_output",
                  axis = "channel",
                  start_index = 1,
                  end_index = 2)

builder.add_slice(name = "slice_h",
                  input_name = "decoded_hw_output",
                  output_name = "slice_h_output",
                  axis = "channel",
                  start_index = 0,
                  end_index = 1)

builder.add_slice(name = "slice_w",
                  input_name = "decoded_hw_output",
                  output_name = "slice_w_output",
                  axis = "channel",
                  start_index = 1,
                  end_index = 2)

builder.add_elementwise(name = "concat",
                        input_names = ["slice_x_output", "slice_y_output", 
                                       "slice_w_output", "slice_h_output"],
                        output_name = "concat_output",
                        mode = "CONCAT")

# (4, num_anchors, 1) --> (1, num_anchors, 4)
builder.add_permute(name = "permute_output",
                    dim = (0, 3, 2, 1),
                    input_name = "concat_output",
                    output_name = "raw_coordinates")


decoder_model = coremltools.models.MLModel(builder.spec)
# decoder_model.save("/Volumes/IPEVO_X0244/object_detection_dataset//Decoder.mlmodel")

In [33]:
nms_spec = coremltools.proto.Model_pb2.Model()
nms_spec.specificationVersion = 3
for i in range(2):
    decoder_output = decoder_model._spec.description.output[i].SerializeToString()

    nms_spec.description.input.add()
    nms_spec.description.input[i].ParseFromString(decoder_output)

    nms_spec.description.output.add()
    nms_spec.description.output[i].ParseFromString(decoder_output)

nms_spec.description.output[0].name = "confidence"
nms_spec.description.output[1].name = "coordinates"

output_sizes = [object_class, 4]
for i in range(2):
    ma_type = nms_spec.description.output[i].type.multiArrayType
    ma_type.shapeRange.sizeRanges.add()
    ma_type.shapeRange.sizeRanges[0].lowerBound = 0
    ma_type.shapeRange.sizeRanges[0].upperBound = 20
    ma_type.shapeRange.sizeRanges.add()
    ma_type.shapeRange.sizeRanges[1].lowerBound = output_sizes[i]
    ma_type.shapeRange.sizeRanges[1].upperBound = output_sizes[i]
    del ma_type.shape[:]

nms = nms_spec.nonMaximumSuppression
nms.confidenceInputFeatureName = "raw_confidence"
nms.coordinatesInputFeatureName = "raw_coordinates"
nms.confidenceOutputFeatureName = "confidence"
nms.coordinatesOutputFeatureName = "coordinates"
nms.iouThresholdInputFeatureName = "iouThreshold"
nms.confidenceThresholdInputFeatureName = "confidenceThreshold"

default_iou_threshold = 0.6
default_confidence_threshold = 0.4
nms.iouThreshold = default_iou_threshold
nms.confidenceThreshold = default_confidence_threshold

nms.pickTop.perClass = True
# labels = np.loadtxt("/Users/kehwaweng/Documents/ObjectDetection/create_custom_dataset/custom_dataset_class_list.txt", 
labels = np.loadtxt("/Volumes/IPEVO_X0244/object_detection_dataset/custom_dataset_class_list_153.txt",
                    dtype=str, delimiter="\n")
nms.stringClassLabels.vector.extend(labels)

nms_model = coremltools.models.MLModel(nms_spec)
# nms_model.save("/Volumes/IPEVO_X0244/object_detection_dataset//NMS.mlmodel")


In [34]:
# input_features = [ ("image", datatypes.Array(3, 300, 300)),
input_features = [ ("Preprocessor/sub", datatypes.Array(3, 300, 300)), #]
                   ("iouThreshold", datatypes.Double()),
                   ("confidenceThreshold", datatypes.Double()) ]

output_features = [ "confidence", "coordinates" ]

pipeline = Pipeline(input_features, output_features)

# We added a dimension of size 1 to the back of the inputs of the decoder 
# model, so we should also add this to the output of the SSD model or else 
# the inputs and outputs do not match and the pipeline is not valid.
# ssd_output = ssd_model._spec.description.output
# ssd_output[0].type.multiArrayType.shape[:] = [4, 1917, 1] #[198 + 1, 1917, 1]
# ssd_output[1].type.multiArrayType.shape[:] = [199, 1917, 1] ##[4, 1917, 1]

pipeline.add_model(ssd_model)
pipeline.add_model(decoder_model)
pipeline.add_model(nms_model)

# The "image" input should really be an image, not a multi-array.
pipeline.spec.description.input[0].ParseFromString(ssd_model._spec.description.input[0].SerializeToString())

# Copy the declarations of the "confidence" and "coordinates" outputs.
# The Pipeline makes these strings by default.
pipeline.spec.description.output[0].ParseFromString(nms_model._spec.description.output[0].SerializeToString())
pipeline.spec.description.output[1].ParseFromString(nms_model._spec.description.output[1].SerializeToString())

# Add descriptions to the inputs and outputs.
pipeline.spec.description.input[1].shortDescription = "(optional) IOU Threshold "
pipeline.spec.description.input[2].shortDescription = "(optional) Confidence Threshold"
pipeline.spec.description.output[0].shortDescription = "Class confidence"
pipeline.spec.description.output[1].shortDescription = "[x, y, width, height]"

# Add metadata to the model.
pipeline.spec.description.metadata.versionString = "ssdlite_mobilenet_v2_coco_2018_05_09"
pipeline.spec.description.metadata.shortDescription = "ssdlite with mobilenetv2, train on COCO Dataset"
pipeline.spec.description.metadata.author = "Converted by IPEVO"
pipeline.spec.description.metadata.license = "https://github.com/tensorflow/models/blob/master/research/object_detection"

# Add the list of class labels and the default threshold values too.
user_defined_metadata = {
    "iou_threshold": str(default_iou_threshold),
    "confidence_threshold": str(default_confidence_threshold),
    "classes": ",".join(labels)
}
pipeline.spec.description.metadata.userDefined.update(user_defined_metadata)

# Don't forget this or Core ML might attempt to run the model on an unsupported
# operating system version!
pipeline.spec.specificationVersion = 3

final_model = coremltools.models.MLModel(pipeline.spec)
# final_model.save("/Volumes/IPEVO_X0244/object_detection_dataset/final_test_model.mlmodel")
final_model.save("/Users/kehwaweng/Documents/ObjectDetection/objectDetectionBeta/objectDetectionBeta/model/final_test_model.mlmodel")
print("done")

done


In [40]:
import shutil
import os

target = "crutch_in_room"

folder = os.path.join("/Users/kehwaweng/Documents/imageCrawler/Image-Downloader/download_images", target)


    
for file in os.listdir(folder):
    if file == ".DS_Store": continue
#     shutil.move(os.path.join(folder, file), os.path.join(folder, file.replace(target, "helicopter"))) 
    shutil.move( os.path.join(folder, file), os.path.join( folder, file.replace(target, target.split("_")[0])))



